In [3]:
import pandas as pd
import numpy as np
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
print("Setup Complete")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Setup Complete


In [4]:
pwd

'/home/ryan/stark/MachineHack/Food_QUalityA_ParticipantsData'

# Load data

In [5]:
# Read the file into a variable fifa_data
filepath = "/home/ryan/stark/MachineHack/Food_QUalityA_ParticipantsData/Data_Train.xlsx"
data = pd.read_excel(filepath)


In [137]:
testdata = pd.read_excel('/home/ryan/stark/MachineHack/Food_QUalityA_ParticipantsData/Data_Test.xlsx')
data.head(3)

,LicenseNo,FacilityID,FacilityName,Type,Street,City,State,LocationID,Reason,SectionViolations,RiskLevel,Geo_Loc,Inspection_Results
0,4744,8123,7715,RESTAURANT,15522,id-11235901,id_1890134,81876.0,CANVASS,33.0,High,locid16406,4
1,2973,12268,11664,GROCERY STORE,3057,id-11235901,id_1890134,81862.0,COMPLAINT,33.0,High,locid878,4
2,18223,1112,969,RESTAURANT,14988,id-11235901,id_1890134,81883.0,CANVASS,NaN,High,locid3368,6


In [7]:
cols_to_drop = [
    'Date',
    'ID'
    ]

 

In [155]:
data['Type'].duplicated().sum()

147033

In [8]:
#cata = data.copy()

In [9]:
# cata['Date']

In [10]:
data = data.drop(cols_to_drop, axis=1)


In [11]:
# from datetime import datetime

In [12]:
# data['Date'] = pd.to_datetime(data['Date'] ,errors='coerce')

In [13]:
# data["Date"].dtype

In [156]:
data.shape

(147443, 13)

In [157]:
147443/147033

1.002788489658784

# Missing Values

In [15]:
total = data.isnull().sum().sort_values(ascending=False)
percent_1 = data.isnull().sum()/data.isnull().count()*100
percent_2 = (np.round(percent_1, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%']) #ptr
missing_data


,Total,%
SectionViolations,39068,26.5
Type,3485,2.4
LocationID,35,0.0
Inspection_Results,0,0.0
Geo_Loc,0,0.0
RiskLevel,0,0.0
Reason,0,0.0
State,0,0.0
City,0,0.0
Street,0,0.0


In [205]:
from sklearn.model_selection import train_test_split
y = data.Inspection_Results

# To keep things simple, we'll use only numerical predictors
melb_predictors = data.drop(['Inspection_Results'], axis=1)
X = melb_predictors.copy() #select_dtypes(exclude=['object']) #since mostly problem is in section violation

# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.999, test_size=0.001,random_state=0)

# score_dataset fun:

In [159]:
#####handler function
#######scoring the dataset#########
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import log_loss
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import classification_report

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestClassifier(random_state=0)
    model.fit(X_train, y_train)
    #preds = model.predict(X_valid)
    #log_loss(y_valid, preds ,labels=[0,1,2,3,4,5,6])
    return model.score(X_valid,y_valid) 

In [160]:
# cols_with_missing = [col for col in X_train.columns
#                      if X_train[col].isnull().any()]

# # Drop columns in training and validation data
# reduced_X_train = X_train.drop(cols_with_missing, axis=1)
# reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

# print("Approach 1 (Drop columns with missing values):")
# print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

In [161]:
# from sklearn.impute import SimpleImputer

# # Imputation
# my_imputer = SimpleImputer(strategy='constant')
# imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
# imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# # Imputation removed column names; put them back
# imputed_X_train.columns = X_train.columns
# imputed_X_valid.columns = X_valid.columns

# print("loss from Approach 2 (Imputation):")
# print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

In [162]:
# from sklearn.impute import SimpleImputer

# # Imputation
# my_imputer = SimpleImputer()
# imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
# imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# # Imputation removed column names; put them back
# imputed_X_train.columns = X_train.columns
# imputed_X_valid.columns = X_valid.columns

# print("loss from Approach 2 (Imputation):")
# print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

In [163]:
# from sklearn.impute import SimpleImputer

# # Imputation
# my_imputer = SimpleImputer(strategy='most_frequent')
# imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
# imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# # Imputation removed column names; put them back
# imputed_X_train.columns = X_train.columns
# imputed_X_valid.columns = X_valid.columns

# print("loss from Approach 2 (Imputation):")
# print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

In [164]:
# X_train_plus = X_train.copy()
# X_valid_plus = X_valid.copy()

# # Make new columns indicating what will be imputed
# for col in cols_with_missing:
#     X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
#     X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

# # Imputation
# my_imputer = SimpleImputer()
# imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
# imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

# # Imputation removed column names; put them back
# imputed_X_train_plus.columns = X_train_plus.columns
# imputed_X_valid_plus.columns = X_valid_plus.columns

# print("score from Approach 3 (An Extension to Imputation):")
# print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))


In [165]:
#########It means second approach is best########

In [166]:
categorical_cols = [cname for cname in X.columns if X[cname].nunique() < 500000000000000000 and 
                        X[cname].dtype == "object"]

In [167]:
categorical_cols

['Type', 'City', 'State', 'Reason', 'RiskLevel', 'Geo_Loc']

In [168]:
numerical_cols = [cname for cname in X.columns if X[cname].dtype in ['int64', 'float64']]


In [169]:
numerical_cols

['LicenseNo',
 'FacilityID',
 'FacilityName',
 'Street',
 'LocationID',
 'SectionViolations']

In [170]:
datecol = ['Date']

In [171]:
my_cols = categorical_cols + numerical_cols
my_cols

['Type',
 'City',
 'State',
 'Reason',
 'RiskLevel',
 'Geo_Loc',
 'LicenseNo',
 'FacilityID',
 'FacilityName',
 'Street',
 'LocationID',
 'SectionViolations']

In [172]:

# Select numerical columns

# # Keep selected columns only
# my_cols = low_cardinality_cols + numerical_cols
# X_train = X[my_cols].copy()
# X_valid = X[my_cols].copy()

In [173]:
####imputing numerical columns###

In [174]:
# nX_train, nX_valid, ny_train, ny_valid = train_test_split(X[numerical_cols], y, train_size=0.8, test_size=0.2,random_state=0)

In [175]:
# from sklearn.impute import SimpleImputer

# # Imputation
# my_imputer = SimpleImputer(strategy='most_frequent')
# imNX_train = pd.DataFrame(my_imputer.fit_transform(nX_train))
# imNX_valid = pd.DataFrame(my_imputer.transform(nX_valid))

# # Imputation removed column names; put them back
# imNX_train.columns = nX_train.columns
# imNX_valid.columns = nX_valid.columns

In [176]:
###################imputing catgorical columns######333

In [177]:
# cX_train, cX_valid, cy_train, cy_valid = train_test_split(X[low_cardinality_cols], y, train_size=0.8, test_size=0.2,random_state=0)

In [178]:
# my_imputer = SimpleImputer(strategy='most_frequent')
# imcX_train = pd.DataFrame(my_imputer.fit_transform(cX_train))
# imcX_valid = pd.DataFrame(my_imputer.transform(cX_valid))

# # Imputation removed column names; put them back
# imcX_train.columns = cX_train.columns
# imcX_valid.columns = cX_valid.columns

In [179]:
# X_train = pd.concat([imNX_train,imcX_train] ,axis=1)

In [180]:
 X_train.head(3)

,LicenseNo,FacilityID,FacilityName,Type,Street,City,State,LocationID,Reason,SectionViolations,RiskLevel,Geo_Loc
67528,26121,7628,7252,RESTAURANT,8568,id-11235901,id_1890134,81846.0,LICENSE,NaN,High,locid7682
74634,36379,9294,8818,RESTAURANT,16553,id-11235901,id_1890134,81865.0,CANVASS,33.0,High,locid1912
42662,20219,4642,4429,SCHOOL,7225,id-11235901,id_1890134,81897.0,CANVASS,21.0,High,locid1352


In [181]:
# X_valid = pd.concat([imNX_valid,imcX_valid] ,axis=1)

In [182]:
 X_valid.head()

,LicenseNo,FacilityID,FacilityName,Type,Street,City,State,LocationID,Reason,SectionViolations,RiskLevel,Geo_Loc
76851,28225,4702,4480,RESTAURANT,13072,id-11235901,id_1890134,81875.0,CANVASS,2.0,High,locid15302
36203,743,25793,24635,CAFE/STORE,2087,id-11235901,id_1890134,81882.0,CANVASS,32.0,Medium,locid3211
3760,24297,13257,12588,RESTAURANT,2382,id-11235901,id_1890134,81855.0,CANVASS RE-INSPECTION,NaN,High,locid9774
51377,18030,7609,7234,SCHOOL,12441,id-11235901,id_1890134,81883.0,CANVASS,32.0,High,locid3875
116892,17544,23678,22588,RESTAURANT,479,id-11235901,id_1890134,81862.0,CANVASS,33.0,High,locid511


# ENCODING

In [183]:
### perdorm suitable imputation selected from above
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer(strategy='most_frequent')
imX_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imX_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removed column names; put them back
imX_train.columns = X_train.columns
imX_valid.columns = X_valid.columns


In [184]:
####convert back to datatypes 
imX_train[numerical_cols] = imX_train[numerical_cols].apply(pd.to_numeric, errors='coerce')
imX_valid[numerical_cols] = imX_valid[numerical_cols].apply(pd.to_numeric, errors='coerce')


In [64]:
print(score_dataset(imX_train, imX_valid, y_train, y_valid))

ValueError: could not convert string to float: 'SCHOOL'

In [185]:
imX_train.dtypes

LicenseNo              int64
FacilityID             int64
FacilityName           int64
Type                  object
Street                 int64
City                  object
State                 object
LocationID           float64
Reason                object
SectionViolations    float64
RiskLevel             object
Geo_Loc               object
dtype: object

In [186]:
imX_valid.dtypes

LicenseNo              int64
FacilityID             int64
FacilityName           int64
Type                  object
Street                 int64
City                  object
State                 object
LocationID           float64
Reason                object
SectionViolations    float64
RiskLevel             object
Geo_Loc               object
dtype: object

In [187]:
# Get list of categorical variables
s = (imX_train.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

Categorical variables:
['Type', 'City', 'State', 'Reason', 'RiskLevel', 'Geo_Loc']


In [199]:

np.where(np.array(X['Geo_Loc']) ==['locid9665'])

(array([123026]),)

In [201]:
X['Geo_Loc'][123026] = 'locid7682'

/home/ryan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [202]:
X['Geo_Loc'][123026]

'locid7682'

In [206]:
from sklearn.preprocessing import LabelEncoder

# Make copy to avoid changing original data 
label_X_train = imX_train.copy()
label_X_valid = imX_valid.copy()

# Apply label encoder to each column with categorical data
label_encoder = LabelEncoder()
for col in object_cols:
    label_X_train[col] = label_encoder.fit_transform(imX_train[col])
    label_X_valid[col] = label_encoder.transform(imX_valid[col])

print(" loss from Approach 1 (Label Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

ValueError: y contains previously unseen labels: 'locid9665'

In [149]:
label_encoder = LabelEncoder()
for col in object_cols:
    label_X_train[col] = label_encoder.fit_transform(X)
    #label_X_valid[col] = label_encoder.transform(imX_valid[col])

ValueError: bad input shape (147443, 12)

In [147]:
label_encoder = LabelEncoder()
for col in object_cols:
    print(X[col])
    

0                           RESTAURANT
1                        GROCERY STORE
2                           RESTAURANT
3                           RESTAURANT
4                           RESTAURANT
                      ...             
147438                   GROCERY STORE
147439                      RESTAURANT
147440                      RESTAURANT
147441                      RESTAURANT
147442    CHILDREN'S SERVICES FACILITY
Name: Type, Length: 147443, dtype: object
0         id-11235901
1         id-11235901
2         id-11235901
3         id-11235901
4         id-11235901
             ...     
147438    id-11235901
147439    id-11235901
147440    id-11235901
147441    id-11235901
147442    id-11235901
Name: City, Length: 147443, dtype: object
0         id_1890134
1         id_1890134
2         id_1890134
3         id_1890134
4         id_1890134
             ...    
147438    id_1890134
147439    id_1890134
147440    id_1890134
147441    id_1890134
147442    id_1890134
Name: State, L